In [11]:
import random
import numpy as np
import math
import time
start_time = time.time()

data = np.genfromtxt('HousePriceDataTRAINING.csv', delimiter=',')
train = data
test = np.genfromtxt('HousePriceDataTEST.csv', delimiter=',')

# Making train and test sets random
# bool_array = []
# choice = [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False]
# for x in range(len(data[:,0])):
#     bool_array.append(random.choice(choice))
# bool_rand = np.array(bool_array)
# train = data[bool_rand]
# test = data[np.invert(bool_rand)]

# test = np.genfromtxt('HousePriceDataTEST.csv', delimiter=',')[:]
train[0,0] = 53.2913325
test[0,0] = 53.6301728

guess = []
actual = []

counter = 1
n = 15 # Nearby houses to count
c = 2 # Deviation co-efficient

def adjust_for_inflation(house_array):
    days_before = 43563 - house_array[: , 2]
    # print(f"Days till 2019:   {days_before}")
    converted_price_array = house_array[:, 3] * (1.000125 ** days_before)
    return converted_price_array

def reverse_inflation(price_estimate, date_equiv):
    days_after = 43563 - date_equiv
    reversed_price = price_estimate / (1.000125 ** days_after)
    # print(f"Days before 2019:   {days_after} : {reversed_price}")
    return reversed_price

def remove_outliers(oneD_array, deviations):
    # w =
    variance = ((oneD_array - np.average(oneD_array)) ** 2 ).sum()/len(oneD_array)
    deviation = math.sqrt(variance)
    outlier_bool = np.logical_and(oneD_array < (np.average(oneD_array) + deviations * deviation), oneD_array > (np.average(oneD_array) - deviations * deviation))
    outliers = oneD_array[outlier_bool]
    return outliers

# Calculates median & average error percentage
def data_accuracy(predictions, real):
    p, r = predictions, real
    w = 1.5
    array = np.stack((p, r), axis=1)
    difference = np.abs(array[:,0]-array[:,1])
    percentages = (difference/np.abs(array[:,1]))*100
    # removes top and bottom 5% respectively
    without_outliers = remove_outliers(percentages, 2)
    without_outliers.sort()
    median = without_outliers[int(len(without_outliers)/2)]

    print(f"average of 90%: {np.average(without_outliers)}")
    print(f"Median is: {median}")

def remove_price_outliers(dist_array, price_array):

    mean = dist_array.sum()/len(dist_array)
    diff_from_mean_sqrd = (dist_array - mean) ** 2
    variance = diff_from_mean_sqrd.sum()/len(dist_array)
    deviation = math.sqrt(variance)
    use_bool = np.logical_and(dist_array < (mean + c * deviation), dist_array > (mean - c * deviation))
    # set array to remove outliers
    in_array = np.stack((dist_array, price_array), axis=1)
    new_array = in_array[use_bool]
    return new_array


for x in range(len(test[:,0])-1):

    # print("--------------------------")
    # print(f"Actual Price: {test[x, 3]}")
    # actual.append(test[x, 3])
    date = test[x, 2]
    # x = 20
    num_rooms = test[x, 4]
    room_bool = (train[:,4] == num_rooms)
    # bool array with all same room number true
    same_num_rooms = train[room_bool]

    # array with just latitude and longitude of array of the same room number
    lat_long = same_num_rooms[:,:2]
    # lat_long[0] = Latitude     lat_long[1] = Longitude
    test_lat = test[x,0]
    test_long = test[x,1]

    distance_list = ((lat_long[:,0] - test_lat) ** 2) + ((lat_long[:,1] - test_long) ** 2)
    # Adjusts prices for inflation while matching them to distances
    to_sort = np.stack((distance_list, adjust_for_inflation(same_num_rooms)), axis=1)
    sorted_array = to_sort[to_sort[:, 0].argsort()]

    no_outliers = remove_price_outliers(sorted_array[:n, 0], sorted_array[:n, 1])

    estimate_inflated = no_outliers[:, 1].sum()/len(no_outliers[:, 1])

    # print(f"Guessed Price: {estimate}")
    # print(f"{counter}.")
    counter += 1
    estimate = reverse_inflation(estimate_inflated, date)
    guess.append(estimate)

# data_accuracy(guess, actual)
# for i in range(len(guess)):
#     print(f"{guess[i]} : {actual[i]}")
print("--- %s seconds ---" % (time.time() - start_time))
answers = np.array(guess)
np.savetxt('data.csv', answers, delimiter=',', axis=1)




IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.genfromtxt('HousePriceDataTRAINING.csv', delimiter=',')
split = 199900 # out of 200,000
train = data[:split]
test = data[split:]
test[0,0] = 53.2115009
train_boolA = train[:, 4] > 3000000
trainA = train[train_boolA]
train_bool = trainA[:, 0] < 53.67530712
expensive_houses = trainA[train_bool]
f = plt.figure()
f.set_figwidth(6)
f.set_figheight(8)
plt.scatter(expensive_houses[:, 1], expensive_houses[:, 0], )
plt.show()

In [ ]:
import numpy as np
import math

data = np.genfromtxt('HousePriceDataTRAINING.csv', delimiter=',')
split = 199000 # out of 200,000
train = data[:split]
test = data[split:]
test[0,0] = 53.2115009

print((data[:, 3].sum())/len(data[:, 3]))

# for x in range(len(test[:,0])-1):
#
#     print("--------------------------")
#     print(f"Actual Price: {test[x, 3]}")
#     # x = 20
#     num_rooms = test[x, 4]
#     room_bool = (train[:,4] == num_rooms)
#     # bool array with all same room number true
#     same_num_rooms = train[room_bool]
#     # array with just latitude and longitude of array of the same room number
#     lat_long = same_num_rooms[:,:2]
#     # lat_long[0] = Latitude     lat_long[1] = Longitude
#     test_lat = test[x,0]
#     test_long = test[x,1]
#
#     distance = ((lat_long[:,0] - test_lat) ** 2) + ((lat_long[:,1] - test_long) ** 2)
#     i = 0.0038
#     i_array = lat_long[:,0]
#     distance_bool = distance < i
#     print(f"Houses Near: {len(i_array[distance_bool])}")